In [1]:
# https://www.kaggle.com/c/bluebook-for-bulldozers

In [2]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from category_encoders import OrdinalEncoder

In [3]:
df = pd.read_csv('Train.csv', parse_dates=["saledate"], low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                

In [4]:
df.isnull().sum().sort_index() / len(df)

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [5]:
df.select_dtypes('number').columns.values

array(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter'],
      dtype=object)

In [6]:
df.select_dtypes('object').columns.values

array(['UsageBand', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting',
       'Blade_Type', 'Travel_Controls', 'Differential_Type',
       'Steering_Controls'], dtype=object)

In [7]:
df[df.select_dtypes('object').columns.values].nunique().sort_index()

Backhoe_Mounting              2
Blade_Extension               2
Blade_Type                   10
Blade_Width                   6
Coupler                       3
Coupler_System                2
Differential_Type             4
Drive_System                  4
Enclosure                     6
Enclosure_Type                3
Engine_Horsepower             2
Forks                         2
Grouser_Tracks                2
Grouser_Type                  3
Hydraulics                   12
Hydraulics_Flow               3
Pad_Type                      4
Pattern_Changer               3
ProductGroup                  6
ProductGroupDesc              6
ProductSize                   6
Pushblock                     2
Ride_Control                  3
Ripper                        4
Scarifier                     2
Steering_Controls             5
Stick                         2
Stick_Length                 29
Thumb                         3
Tip_Control                   3
Tire_Size                    17
Track_Ty

In [8]:
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_week'] = df['saledate'].dt.week
df['sale_day'] = df['saledate'].dt.day
df['sale_dayofweek'] = df['saledate'].dt.dayofweek
df['sale_dayofyear'] = df['saledate'].dt.dayofyear
df['sale_is_month_start'] = df['saledate'].dt.is_month_start
df['sale_is_month_end'] = df['saledate'].dt.is_month_end
df['sale_is_quarter_start'] = df['saledate'].dt.is_quarter_start
df['sale_is_quarter_end'] = df['saledate'].dt.is_quarter_end
df['sale_is_year_start'] = df['saledate'].dt.is_year_start
df['sale_is_year_end'] = df['saledate'].dt.is_year_end
df['sale_elapsed'] = (df['saledate'] - df['saledate'].min()).dt.days

In [9]:
df['SalePrice'] = np.log(df['SalePrice'])

In [10]:
# test size is the same from Kaggle
X_train, X_test, y_train, y_test = train_test_split(df, df['SalePrice'], test_size=12000, shuffle=False)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((389125, 66), (389125,), (12000, 66), (12000,))

In [11]:
class MissingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.isnull().values

numerical_columns = np.setdiff1d(df.select_dtypes('number').columns.values, ['SalePrice'])
categorical_columns = df.select_dtypes('object').columns.values
na_columns = df.columns[df.isna().any()].values

pipeline = make_pipeline(
    make_column_transformer(
        (numerical_columns, SimpleImputer(strategy='median')),
        (categorical_columns, OrdinalEncoder()),
        (na_columns, MissingTransformer()),
    ),
    RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)
)

pipeline.fit(df, df['SalePrice'])
pipeline.score(df, df['SalePrice'])

0.9825654751182998

In [12]:
import math

def rmse(x, y): return math.sqrt(((x - y)**2).mean())

pipeline.fit(X_train, y_train)

(
    rmse(pipeline.predict(X_train), y_train),
    rmse(pipeline.predict(X_test), y_test),
    pipeline.score(X_train, y_train),
    pipeline.score(X_test, y_test),
)

(0.09167419248700492,
 0.2551825742187419,
 0.9824357741678488,
 0.8837079564281558)